## Political Corruption News Filtering Script

### Overview
This script processes datasets of news articles from multiple countries to identify those related to political corruption. It applies multiple filtering steps based on country-specific keywords and text similarity using TF-IDF and cosine similarity. The final filtered datasets are saved for further inductive analysis of frame identification.

---

### Steps

#### 1. **Load Data**
- The script reads CSV files containing news articles from specified datasets (Netherlands, Italy, Bulgaria, United Kingdom).
- It prints the total number of articles in each original dataset.

#### 2. **Filter on Selected Outlets**
- For each country, a list of selected news outlets is loaded from a text file.
- Articles are filtered to keep only those from the selected outlets.
- If no selection file exists for a country, outlet filtering is skipped with a warning.

#### 3. **Filter on Keywords**
- The function `is_political_corruption()` checks if an article contains **both** political and corruption-related terms specific to each country.
- Articles meeting this condition are marked as relevant.
- The dataset is filtered to keep only relevant articles.

#### 4. **TF-IDF Similarity Filter**
- Country-specific reference texts containing political corruption phrases are used as benchmarks.
- TF-IDF (Term Frequency-Inverse Document Frequency) vectorization transforms article bodies and reference texts into numerical vectors.
- Cosine similarity is computed between each article and the reference texts.
- Articles with a similarity score above 0.1 are retained to improve precision.

#### 5. **Export Filtered Data**
- The filtered datasets are saved as both CSV and Excel files in the configured output folder.

#### 6. **Stratified Date Sampling**
- Article `dateTime` fields are converted to datetime objects and timezone information is removed.
- A `month` period column is created for grouping.
- Stratified sampling is performed by selecting up to 3 articles per month, then sampling a total of up to 30 articles.
- The sampled subset is saved as an Excel file for manual inductive analysis.

---

### Enhancements & Multi-Country Extension
- **Country-specific keyword lists** for political and corruption terms ensure contextual accuracy.
- **Outlet filtering** enables focus on trusted or relevant media sources.
- **TF-IDF with cosine similarity** enables semantic filtering beyond keyword matching.
- **Stratified sampling by month** balances the temporal distribution of selected articles.
- The output folder structure and naming conventions are consistent and organized by country.

---

### Output Files

All output files are saved to: `~/webdav/ASCOR-FMG-5580-RESPOND-news-data (Projectfolder)/output/data-inductive-analysis/FILTERED_SAMPLES/`


For each country (`Netherlands`, `Italy`, `Bulgaria`, `United_Kingdom`), the following files are created:

| File Type        | Filename Pattern               | Description                     |
|------------------|--------------------------------|---------------------------------|
| CSV              | `filtered_<country>.csv`       | Full filtered dataset            |
| Excel            | `filtered_<country>.xlsx`      | Full filtered dataset            |
| Excel Sample     | `sampled_<country>.xlsx`       | Stratified monthly sample (~30) |

---

If you want me to generate a more concise README or add usage instructions, just say!




In [ ]:
import pandas as pd
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

# ========== CONFIGURATION ==========

DATASETS = {
    "Netherlands": "~/webdav/ASCOR-FMG-5580-RESPOND-news-data (Projectfolder)//Netherlands_news.csv",
    "United_Kingdom": "~/webdav/ASCOR-FMG-5580-RESPOND-news-data (Projectfolder)/United_Kingdom_news.csv",
    "Bulgaria": "~/webdav/ASCOR-FMG-5580-RESPOND-news-data (Projectfolder)/Bulgaria_news.csv",
    "Italy": "~/webdav/ASCOR-FMG-5580-RESPOND-news-data (Projectfolder)/Italy_news.csv"
}

OUTPUT_FOLDER = "~/webdav/ASCOR-FMG-5580-RESPOND-news-data (Projectfolder)/output/data-inductive-analysis/FILTERED_SAMPLES/"
OUTLET_SELECTION_FOLDER = "~/webdav/ASCOR-FMG-5580-RESPOND-news-data (Projectfolder)/selected_outlets/"

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# ========== COUNTRY-SPECIFIC TERMS ==========
country_terms = {
    "Netherlands": {
        "political": [
            "politieke partij", "minister", "kabinet", "regering", "parlement",
            "staatscorruptie", "partijfinanciering", "burgemeester", "wethouder",
            "president", "dictator", "politieke leider", "stemfraude", "verkiezingsfraude",
            "wetgevende macht", "politiek schandaal", "overheidscontracten", "ambtsmisbruik"
        ],
        "corruption": [
            "omkoping", "belangenverstrengeling", "nepotisme", "machtsmisbruik",
            "staatsfraude", "stemfraude", "corruptie binnen overheid", "ministeriële omkoping",
            "partijfinanciering", "politieke afpersing", "parlementaire fraude",
            "regeringsfraude", "wangedrag", "witwassen", "steekpenningen", "vriendjespolitiek"
        ]
    },
    "Italy": {
        "political": [
            "partito politico", "ministro", "governo", "parlamento", "presidente",
            "leader politico", "elezioni", "scandalo politico", "corruzione di stato"
        ],
        "corruption": [
            "corruzione", "tangenti", "frode", "abuso di potere", "clientelismo",
            "finanziamento illecito", "lavaggio di denaro", "appropriazione indebita"
        ]
    },
    "Bulgaria": {
        "political": [
            "политическа партия", "министър", "правителство", "парламент",
            "кмет", "президент", "политически лидер", "избори",
            "законодателна власт", "политически скандал", "народен представител"
        ],
        "corruption": [
            "корупция", "подкуп", "измама", "злоупотреба с власт", "непотизъм",
            "шуробаджанащина", "пране на пари", "конфликт на интереси",
            "клиентелизъм", "финансиране на партия", "присвояване"
        ]
    },
    "United_Kingdom": {
        "political": [
            "political party", "minister", "cabinet", "government", "parliament",
            "mayor", "prime minister", "elections", "political scandal"
        ],
        "corruption": [
            "corruption", "bribery", "fraud", "influence peddling", "abuse of power",
            "cronyism", "money laundering", "conflict of interest", "embezzlement"
        ]
    }
}

reference_texts = {
    "Netherlands": [
        "politieke corruptie omkoping fraude belangenverstrengeling machtsmisbruik",
        "corruptie binnen de overheid, ministeriële omkoping, partijfinanciering schandalen",
        "staatsfraude, verkiezingsfraude, corruptie binnen politieke partijen",
        "politiek schandaal, stemfraude, vriendjespolitiek in de regering"
    ],
    "Italy": [
        "corruzione, frode, abuso di potere, clientelismo, scandalo politico, appropriazione indebita, tangenti",
        "finanziamento illecito ai partiti, lavaggio di denaro, scandali politici",
        "corruzione governativa, nepotismo, abuso d'ufficio"
    ],
    "Bulgaria": [
        "corruption, bribery, fraud, abuse of power, political scandal, money laundering",
        "nepotism, cronyism, party financing, election fraud",
        "embezzlement, misuse of public funds, political misconduct",
        "conflict of interest, corruption in parliament, political influence"
    ],
    "United_Kingdom": [
        "corruption, bribery, fraud, influence peddling, political scandal, abuse of power, collusion",
        "misuse of public office, parliamentary fraud, party donations scandal",
        "conflict of interest, cronyism, lobbying corruption"
    ]
}

# ========== LOAD OUTLET SELECTIONS ==========
selected_outlets = {}
for country in DATASETS:
    outlet_file = os.path.join(OUTLET_SELECTION_FOLDER, f"{country}_outlets_selection.txt")
    if os.path.exists(outlet_file):
        with open(outlet_file, encoding="utf-8") as f:
            outlets = {line.strip().lower() for line in f if line.strip()}
        selected_outlets[country] = outlets
    else:
        print(f"⚠️ Warning: No outlet selection file found for {country}. Skipping outlet filtering.")
        selected_outlets[country] = None

# ========== FILTER FUNCTION ==========
def is_political_corruption(text, political_terms, corruption_terms):
    text = text.lower()
    has_political = any(term in text for term in political_terms)
    has_corruption = any(term in text for term in corruption_terms)
    return has_political and has_corruption

# ========== PROCESS DATASETS ==========
for country, file_path in DATASETS.items():
    if not os.path.exists(file_path):
        print(f"Skipping {country}: File not found.")
        continue

    df = pd.read_csv(file_path)
    print(f"\nProcessing {country}: {len(df)} articles")

    # Outlet filtering
    if selected_outlets[country]:
        df["source.uri"] = df["source.uri"].astype(str).str.strip().str.lower()
        df = df[df["source.uri"].isin(selected_outlets[country])]
        print(f"📰 {country}: {len(df)} articles after outlet filtering")

    political_terms = country_terms[country]["political"]
    corruption_terms = country_terms[country]["corruption"]

    df[f"is_political_corruption_{country}"] = df["body"].astype(str).apply(
        lambda x: is_political_corruption(x, political_terms, corruption_terms)
    )

    filtered_df = df[df[f"is_political_corruption_{country}"]].copy()

    # TF-IDF Filtering
    vectorizer = TfidfVectorizer(max_features=5000, token_pattern=r"(?u)\b\w+\b")
    tfidf_matrix = vectorizer.fit_transform(filtered_df["body"])
    reference_vec = vectorizer.transform(reference_texts[country])

    similarity_scores = cosine_similarity(tfidf_matrix, reference_vec).max(axis=1)
    filtered_df["similarity"] = similarity_scores
    final_df = filtered_df[filtered_df["similarity"] > 0.1].drop_duplicates(subset=["body"])

    print(f"✅ {country}: {len(final_df)} articles after similarity filtering")

    # Save full filtered articles
    final_df.to_csv(os.path.join(OUTPUT_FOLDER, f"filtered_{country}.csv"), index=False)
    final_df.to_excel(os.path.join(OUTPUT_FOLDER, f"filtered_{country}.xlsx"), index=False)

    # Stratified sampling
    final_df["dateTime"] = pd.to_datetime(final_df["dateTime"], errors='coerce').dt.tz_localize(None)
    final_df["month"] = final_df["dateTime"].dt.to_period("M")

    monthly_sampled = final_df.groupby("month", group_keys=False).apply(
        lambda x: x.sample(n=min(len(x), 3), random_state=42)
    ).reset_index(drop=True)

    sample_df = monthly_sampled.sample(n=min(len(monthly_sampled), 30), random_state=42)
    sample_df.to_excel(os.path.join(OUTPUT_FOLDER, f"sampled_{country}.xlsx"), index=False)

    print(f"📦 {country}: Sample saved with {len(sample_df)} articles")

## Helper Function to Extract the Outlets
### Overview
This section of the code is responsible for extracting and saving the unique outlets (news sources) from the original datasets of various countries. The goal is to gather the list of national and regional news outlets that will be used in subsequent analyses. These outlets are saved to `.txt` files for easy reference.

### Workflow
1. **Load the Original Datasets:** 
   - The function iterates through the `DATASETS` dictionary, which contains the paths to the original datasets for each country.
   - If a dataset file is missing, the code skips processing that country and prints a message indicating the missing file.

2. **Extract Outlets:**
   - For each country, the code attempts to access the `source.uri` column (or the appropriate column containing the outlet names) from the dataset.
   - It checks if the `source.uri` column exists to ensure that the correct data is available.
   - The code then extracts the unique outlets from this column while discarding any `NaN` values. This creates a clean list of outlets.

3. **Save the Outlets:**
   - The unique outlets are written to a `.txt` file for each country. 
   - Each outlet is saved on a separate line in the file. The output files are stored in the `OUTPUT_FOLDER`, with the filename format being `{country}_outlets.txt`.

4. **Output and Logging:**
   - After processing each country, a success message is printed, showing the path where the outlets were saved.
   - If the `source.uri` column is missing in a dataset, an error message is printed to indicate this issue.

In [ ]:

# ========== LOAD ORIGINAL DATASETS AND SAVE OUTLETS TO .TXT FILES ==========

for country, file_path in DATASETS.items():
    if not os.path.exists(file_path):
        print(f"Skipping {country}: Original file not found.")
        continue
    
    # Load the original dataset
    df = pd.read_csv(file_path)
    
    # Check if 'source.uri' column exists (or adjust to the actual name of the column)
    if 'source.uri' in df.columns:
        # Get the unique set of outlets (ignoring any NaN values)
        outlets = set(df['source.uri'].dropna().unique())
        
        # Define the output file path for each country
        output_file_path = os.path.join(OUTPUT_FOLDER, f"{country}_outlets.txt")
        
        # Write the set of outlets to a text file
        with open(output_file_path, 'w') as file:
            for outlet in outlets: 
                file.write(f"{outlet}\n")
        
        print(f"✅ Outlets for {country} saved to {output_file_path}")
    else:
        print(f"Outlets column not found in {country} dataset.")